In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit

In [2]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix

        for i in range(0, len(self.sig_scattering_matrix)):
            self.sig_s.append(sum(self.sig_scattering_matrix[i]))
            
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_number_of_groups(self):
        
        return self.number_groups
        

In [3]:
energy_groups_2 = [0, 6.25, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [4]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self): 
        
        return self.energy_group
     #   res = next(x for x, val in enumerate(self.energy_groups)
      #                            if val > self.energy)        
       # return res - 1
    
    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [5]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [6]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [7]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [8]:
import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [9]:
def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [10]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [11]:
class Multigroup_estimator:
    def __init__(self):
        self.groups_before = []
        self.weights_before = []
        self.groups_after = []
        self.weights_after = []
        
    def get_flux_ration_n_m(self, first, secound):
        
        number_before = 0.
        number_after = 0.
        for i in range(0, len(self.groups_before)):
            
            if self.groups_before[i] == first:
                number_before += self.weights_before[i]           
                if self.groups_after[i] == secound:
                    number_after += self.weights_after[i]
        
        flux_ratio = number_after / number_before
    
        return flux_ratio   

In [12]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [13]:
def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False

In [14]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [15]:
def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx)     

In [16]:
def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    

In [17]:
def process_real_collision(particle, free_path, c_s, flux_group_estimator):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':
        process_scattering(particle, c_s)

        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        

In [18]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [19]:
def process_one_particle_history(particle, universe, estimators, flux_estimator):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    while not particle.is_terminated():
        
        energy_group_before = particle.get_energy_group()
        weight_before = particle.get_weight()
        

        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        flux_estimator.groups_before.append(particle.get_energy_group())
        flux_estimator.weights_before.append(particle.get_weight())
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions 
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs, flux_estimator)
            
    
    return particle, sum_collisions

In [20]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [21]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])  

In [22]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [23]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [24]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [25]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [26]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [27]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   

In [28]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [29]:
class Flux_group_estimator:
    def __init__(self, group):
        self.group = group
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if particle.get_energy_group() == self.group:
            self.collision_sum[-1]  += particle.get_weight()    

In [30]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    flux_group_estimators = []
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    number_of_groups = universe.xs[0].get_number_of_groups()
    for i in range(0, number_interations):
        print("i == " + str(i))
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []
        flux_group_estimator = Multigroup_estimator()
        for j in range(0, batch_size):
            particle = initial_sources[j]           
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, 
                                                                            estimators, flux_group_estimator)
            batch_particles.append(terminate_particle)
            
            
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        flux_group_estimators.append(flux_group_estimator)
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s)              

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)
     
    return k_effective, k_effective_exp, k_effective_std, estimators, flux_group_estimators

In [31]:
cs_fission_pu_239 = [0.006, 0.060, 0.90, 0.90, 0.060, 0.006]

cs_capture_pu_239 = [0.006, 0.040, 0.20, 0.20, 0.040, 0.006]

cs_scattering_pu_239 = [[0., 0., 0., 0.033, 0.171, 0.024], 
                        [0., 0., 0., 0.275, 0.6, 0.], 
                        [0., 0., 0., 2., 0., 0.],
                        [0, 0., 2., 0., 0., 0.],
                        [0.0, 0.6, 0.275, 0., 0., 0],
                        [0.024, 0.171, 0.033, 0., 0., 0.]]

cs_total_pu_239 = [0.240, 0.975, 3.10, 3.10, 0.975, 0.240]

cs_production_neutrons_pu_239 = [3., 2.5 , 2., 2., 2.5, 3.]

cs_xi_table_pu_239 = [0.48, 0.02, 0., 0., 0.02, 0.48]


pu_239 = Sig(6, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]

In [32]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 1000)

y_m5 = Plane(0, 1, 0, -1000)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective", "reflective", "reflective", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -1000., -1000, 1000, 1000., 1000 )
slab.set_zero_point(0., 0., 0.)

In [35]:
cells = [slab]
materials = [pu_239]
U235_Reactor = Universe(cells, materials)

In [36]:
zero_group_estimator = Flux_group_estimator(0)
first_group_estimator = Flux_group_estimator(1)
third_group_estimator = Flux_group_estimator(2)

group4_estimator = Flux_group_estimator(3)
group5_estimator = Flux_group_estimator(4)
group6_estimator = Flux_group_estimator(5)
estimators = [zero_group_estimator, first_group_estimator, third_group_estimator,
             group4_estimator, group5_estimator, group6_estimator]

In [81]:
test_number_of_particles = 20
test_number_interations = 1000
test_number_inactive = 200

In [82]:
k_effective,k_effective_exp, k_effective_std, estimators_result, flux_group_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, pu_239, estimators)

 num1  6
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 1.5944087585580977   1.5984451535518118  0.0011043114120656394
i == 258
 keff_cycle , k_effective_exp, std_k_effective 1.5997505492314872   1.5984676603738752  0.0010853379198721184
i == 259
 keff_cycle , k_effective_exp, std_k_effective 1.6148043122451972   1.5987445527784738  0.0011021329369934694
i == 260
 keff_cycle , k_effective_exp, std_k_effective 1.6098960656372026   1.5989304113261193  0.0010994319033839908
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.6031508607213443   1.598999599021123  0.0010834695579712056
i == 262
 keff_cycle , k_effective_exp, std_k_effective 1.6042848962554026   1.5990848457507083  0.0010692545736824127
i == 263
 keff_cycle , k_effective_exp, std_k_effective 1.5958545373782484   1.5990335710146375  0.0010533940545564059
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.5952479839446   1.598974421216668  0.0010384900158194271
i == 265
 keff_cycle , k_effective_exp, std_k_effective 1.6089

 keff_cycle , k_effective_exp, std_k_effective 1.5961735232587688   1.5994440841490243  0.0007616197156701853
i == 328
 keff_cycle , k_effective_exp, std_k_effective 1.6099065264232655   1.5995258219792918  0.0007600540473909406
i == 329
 keff_cycle , k_effective_exp, std_k_effective 1.6000256629153107   1.59952969671523  0.0007541490940483933
i == 330
 keff_cycle , k_effective_exp, std_k_effective 1.6017584848826179   1.5995468412395943  0.0007485218309747487
i == 331
 keff_cycle , k_effective_exp, std_k_effective 1.6086666262886007   1.599616457919358  0.000746041181925
i == 332
 keff_cycle , k_effective_exp, std_k_effective 1.6076001519000152   1.5996769404495144  0.0007428341604889565
i == 333
 keff_cycle , k_effective_exp, std_k_effective 1.5979513747764953   1.5996639662715217  0.0007373419396560913
i == 334
 keff_cycle , k_effective_exp, std_k_effective 1.6121532936612875   1.5997571702072664  0.0007377300218891232
i == 335
 keff_cycle , k_effective_exp, std_k_effective 1.604040

 keff_cycle , k_effective_exp, std_k_effective 1.6042121011372072   1.5996078495550619  0.0006240787494872742
i == 398
 keff_cycle , k_effective_exp, std_k_effective 1.6153413832974464   1.59968731184669  0.000625982793421564
i == 399
 keff_cycle , k_effective_exp, std_k_effective 1.5934880839271   1.5996561599475967  0.0006236077795550785
i == 400
 keff_cycle , k_effective_exp, std_k_effective 1.5875183559123491   1.5995954709274205  0.0006234428227707423
i == 401
 keff_cycle , k_effective_exp, std_k_effective 1.6007108578719158   1.599601020116199  0.0006203581824951348
i == 402
 keff_cycle , k_effective_exp, std_k_effective 1.5873292880847225   1.5995402689675282  0.0006202617490230318
i == 403
 keff_cycle , k_effective_exp, std_k_effective 1.610473545392953   1.599594127472087  0.000619544175421844
i == 404
 keff_cycle , k_effective_exp, std_k_effective 1.617194950791399   1.5996804060177698  0.0006225077386291041
i == 405
 keff_cycle , k_effective_exp, std_k_effective 1.6063322528

 keff_cycle , k_effective_exp, std_k_effective 1.603559547319561   1.5997375381152543  0.0005341730273397927
i == 468
 keff_cycle , k_effective_exp, std_k_effective 1.6018030007501756   1.5997452450653846  0.0005322319147606419
i == 469
 keff_cycle , k_effective_exp, std_k_effective 1.5962398307845302   1.59973221378553  0.0005304097686315199
i == 470
 keff_cycle , k_effective_exp, std_k_effective 1.6027069521707207   1.599743231335105  0.0005285564772389663
i == 471
 keff_cycle , k_effective_exp, std_k_effective 1.612016698911837   1.5997885208833584  0.0005285464096994214
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.5961248541241015   1.599775051520273  0.0005267718706320686
i == 473
 keff_cycle , k_effective_exp, std_k_effective 1.6010123264185983   1.5997795836627577  0.0005248583241801543
i == 474
 keff_cycle , k_effective_exp, std_k_effective 1.589899577841559   1.5997435252473517  0.000524180974625153
i == 475
 keff_cycle , k_effective_exp, std_k_effective 1.6094238

...

 keff_cycle , k_effective_exp, std_k_effective 1.6007667660306908   1.6000283266963369  0.000385425851923774
i == 676
 keff_cycle , k_effective_exp, std_k_effective 1.5958377511528705   1.6000195229662035  0.000384716025607957
i == 677
 keff_cycle , k_effective_exp, std_k_effective 1.5915563887137059   1.600001780546387  0.0003843184120327926
i == 678
 keff_cycle , k_effective_exp, std_k_effective 1.6083662437452877   1.6000192794233723  0.00038391256583482537
i == 679
 keff_cycle , k_effective_exp, std_k_effective 1.606365703600854   1.6000325287431583  0.0003833392757727241
i == 680
 keff_cycle , k_effective_exp, std_k_effective 1.6062322411420442   1.600045444810656  0.00038275780546985987
i == 681
 keff_cycle , k_effective_exp, std_k_effective 1.6002172243986519   1.6000458019407766  0.00038196138924153446
i == 682
 keff_cycle , k_effective_exp, std_k_effective 1.5911448068787188   1.600027335146042  0.00038161519099464916
i == 683
 keff_cycle , k_effective_exp, std_k_effective 1.5

 keff_cycle , k_effective_exp, std_k_effective 1.6121429813747827   1.5998624781584456  0.00036123119701565975
i == 745
 keff_cycle , k_effective_exp, std_k_effective 1.593960981126405   1.5998516497235244  0.00036073033936194146
i == 746
 keff_cycle , k_effective_exp, std_k_effective 1.5836431262225978   1.5998219637830466  0.0003612907131382388
i == 747
 keff_cycle , k_effective_exp, std_k_effective 1.58353598205919   1.5997921905075  0.00036185655443290884
i == 748
 keff_cycle , k_effective_exp, std_k_effective 1.614840195201045   1.5998196503700797  0.0003622379414438419
i == 749
 keff_cycle , k_effective_exp, std_k_effective 1.6021655816848697   1.5998239234690137  0.000361602774038883
i == 750
 keff_cycle , k_effective_exp, std_k_effective 1.6049475163510751   1.5998332390924357  0.00036106490913662056
i == 751
 keff_cycle , k_effective_exp, std_k_effective 1.609254968031871   1.5998503384190046  0.00036081442733653236
i == 752
 keff_cycle , k_effective_exp, std_k_effective 1.595

 keff_cycle , k_effective_exp, std_k_effective 1.592381367922637   1.599897765473016  0.0003420660190992093
i == 815
 keff_cycle , k_effective_exp, std_k_effective 1.5908540455086855   1.5998830602373018  0.000341825814972588
i == 816
 keff_cycle , k_effective_exp, std_k_effective 1.5927924013493333   1.5998715494274187  0.0003414645223256237
i == 817
 keff_cycle , k_effective_exp, std_k_effective 1.586306088897728   1.599849563267727  0.0003416188808929662
i == 818
 keff_cycle , k_effective_exp, std_k_effective 1.6088414739941934   1.5998641132850837  0.0003413758657082016
i == 819
 keff_cycle , k_effective_exp, std_k_effective 1.5946667649096797   1.5998557169226033  0.0003409273323109379
i == 820
 keff_cycle , k_effective_exp, std_k_effective 1.5928734126038266   1.599844455141444  0.00034056325915079983
i == 821
 keff_cycle , k_effective_exp, std_k_effective 1.6059884416630312   1.5998543488395462  0.0003401583181985285
i == 822
 keff_cycle , k_effective_exp, std_k_effective 1.6077

 keff_cycle , k_effective_exp, std_k_effective 1.6111680656691174   1.6000883540880704  0.0003249318874462098
i == 884
 keff_cycle , k_effective_exp, std_k_effective 1.60844721285923   1.6001005746418293  0.0003246865531930944
i == 885
 keff_cycle , k_effective_exp, std_k_effective 1.6028732231665674   1.6001046223039093  0.00032423747748808505
i == 886
 keff_cycle , k_effective_exp, std_k_effective 1.593471593454715   1.6000949531656452  0.0003239088340174406
i == 887
 keff_cycle , k_effective_exp, std_k_effective 1.618238874908102   1.6001213635320823  0.0003245134899816024
i == 888
 keff_cycle , k_effective_exp, std_k_effective 1.6052308071399257   1.6001287900489543  0.0003241265609355023
i == 889
 keff_cycle , k_effective_exp, std_k_effective 1.5828748369569114   1.60010374802705  0.0003246231235988564
i == 890
 keff_cycle , k_effective_exp, std_k_effective 1.5892755331855637   1.6000880549620624  0.0003245319628153523
i == 891
 keff_cycle , k_effective_exp, std_k_effective 1.6073

 keff_cycle , k_effective_exp, std_k_effective 1.5841180668804185   1.6001081292358326  0.00031132659129062265
i == 953
 keff_cycle , k_effective_exp, std_k_effective 1.61697979134061   1.6001305351616024  0.00031171916373032223
i == 954
 keff_cycle , k_effective_exp, std_k_effective 1.5896315270753456   1.6001166107476952  0.00031161672622966177
i == 955
 keff_cycle , k_effective_exp, std_k_effective 1.6053316198420349   1.6001235180445088  0.00031128036087985696
i == 956
 keff_cycle , k_effective_exp, std_k_effective 1.6022107132660226   1.6001262788847488  0.0003108806010423258
i == 957
 keff_cycle , k_effective_exp, std_k_effective 1.6107393084376174   1.6001402987388476  0.0003107860404504614
i == 958
 keff_cycle , k_effective_exp, std_k_effective 1.6054776777558435   1.6001473401359676  0.0003104556243746326
i == 959
 keff_cycle , k_effective_exp, std_k_effective 1.5934724838370407   1.600138545858894  0.00031017101922974224
i == 960
 keff_cycle , k_effective_exp, std_k_effective

In [105]:
first_group_flux = []
secound_group_flux = []
third_group_flux = []

group4_flux = []
group5_flux = []
group6_flux = []

cs_total_pu_239 = [0.240, 0.975, 3.10, 3.10, 0.975, 0.240]

for i in range(200, len(estimators_result[0].collision_sum) - 1):
    first_group_flux.append(estimators_result[0].collision_sum[i]/(cs_total_pu_239[0]))

for i in range(200, len(estimators_result[1].collision_sum) - 1):
    secound_group_flux.append(estimators_result[1].collision_sum[i]/(cs_total_pu_239[1]))

for i in range(200, len(estimators_result[2].collision_sum) - 1):
    third_group_flux.append(estimators_result[2].collision_sum[i]/(cs_total_pu_239[2]))
    
    
for i in range(200, len(estimators_result[3].collision_sum) - 1):
    group4_flux.append(estimators_result[3].collision_sum[i]/(cs_total_pu_239[3]))

for i in range(200, len(estimators_result[4].collision_sum) - 1):
    group5_flux.append(estimators_result[4].collision_sum[i]/(cs_total_pu_239[4]))

for i in range(200, len(estimators_result[5].collision_sum) - 1):
    group6_flux.append(estimators_result[5].collision_sum[i]/(cs_total_pu_239[5]))
    

average_flux_difference_1 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_1.append(secound_group_flux[i] / first_group_flux[i])
    
average_flux_difference_2 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_2.append(third_group_flux[i] / secound_group_flux[i])
    
average_flux_difference_3 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_3.append(third_group_flux[i] / first_group_flux[i])
    
average_flux_difference_4 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_4.append(group5_flux[i] /group6_flux[i])
    
average_flux_difference_5 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_5.append(group4_flux[i] / group6_flux[i])
    
average_flux_difference_6 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_6.append(group4_flux[i] / group5_flux[i])

    
std_flux_1 = get_std(average_flux_difference_1)
flux_difference_1 = sum(average_flux_difference_1)/len(average_flux_difference_1)

std_flux_2 = get_std(average_flux_difference_2)
flux_difference_2 = sum(average_flux_difference_2)/len(average_flux_difference_2)

std_flux_3 = get_std(average_flux_difference_3)
flux_difference_3 = sum(average_flux_difference_3)/len(average_flux_difference_3)


std_flux_4 = get_std(average_flux_difference_4)
flux_difference_4 = sum(average_flux_difference_4)/len(average_flux_difference_4)

std_flux_5 = get_std(average_flux_difference_5)
flux_difference_5 = sum(average_flux_difference_5)/len(average_flux_difference_5)

std_flux_6 = get_std(average_flux_difference_6)
flux_difference_6 = sum(average_flux_difference_6)/len(average_flux_difference_6)

In [106]:
print(average_flux_difference_4)

[0.4806188650287858, 0.48812343306450323, 0.49996146648144096, 0.5065626918576679, 0.46102093054242443, 0.49771267548585507, 0.45798543168701805, 0.44721475016998796, 0.5015279240478563, 0.46758011116545745, 0.466965732493996, 0.4897029528987381, 0.47236588140949987, 0.4777514127890825, 0.4885351868262832, 0.4813762301435226, 0.4932796869101505, 0.49815470927904687, 0.4803423859812391, 0.46118746611472905, 0.44742830392635763, 0.4653567299621209, 0.5015994243835472, 0.4489359986829114, 0.483955414990258, 0.45246007669414207, 0.5028732706381485, 0.4687916745596713, 0.47489034007913766, 0.49916948679656187, 0.49815769907423163, 0.43058505943952025, 0.5191623789934703, 0.47354938896363385, 0.4863976434828582, 0.4698046109037066, 0.47255693721801956, 0.5103976570624131, 0.46574182944611375, 0.4742470895934762, 0.49736881122467375, 0.47864904005933856, 0.4608740735511713, 0.4678200607093473, 0.4768664909251361, 0.43196134435520855, 0.46112554151464524, 0.44352837669279854, 0.507386204149331

In [107]:
difference = (flux_difference_1 - 0.480) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_1 * 100000))

group 1 to group 2 flux difference flux == 13.235125160165895with std   33.557784709515424


In [108]:
difference = (flux_difference_2 - 0.3125) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_2 * 100000))

group 1 to group 2 flux difference flux == 30.11346300023643with std   20.77279915202315


In [109]:
difference = (flux_difference_3 - 0.150) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_3 * 100000))

group 1 to group 2 flux difference flux == 7.488527369031717with std   9.548357845478476


In [111]:
print(flux_difference_4)

0.48411736274880834


In [112]:
difference = (flux_difference_4 - 0.480) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_4 * 100000))

group 1 to group 2 flux difference flux == 411.7362748808362with std   262.2910441144333


In [113]:
difference = (flux_difference_5 - 0.150) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_5 * 100000))

group 1 to group 2 flux difference flux == 128.50772191167758with std   75.77647721333953


In [114]:
difference = (flux_difference_6 - 0.3125) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_6 * 100000))

group 1 to group 2 flux difference flux == 37.466906054567374with std   20.314041384375276


In [115]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [116]:
file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)

In [117]:
def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()

In [118]:
file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)
